<a href="https://colab.research.google.com/github/xlzuvekas/Machine-Learning/blob/main/COMP_3703_Assignment_7_Word_Vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 7: Word Vectors
Course: COMP 3703 - Natural Language Processing

Due: Friday, March 3 at 11:59pm Mountain time

---

YOUR NAME HERE: Xavier Zuvekas

---

# Preface

The focus of this assignment will be the term-document matrix and TF-IDF weighted values. Word2Vec was also covered this past week, but for the sake of scaling the assignment length appropriately we'll leave that out for now. Regardless, by the end of the assignment you will be able to compute and compare word vectors as discussed in lecture.

As always, to start, let's import and download the necessary packages.

In [1]:
### RUN THIS ###
import nltk
import numpy as np
from numpy.linalg import norm
from math import log

nltk.download('punkt')
nltk.download('gutenberg')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import gutenberg

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Notice the corpus we are using here is from [Project Gutenberg](https://www.gutenberg.org/), an online resource of thousands of e-books. Only a handful of texts are included in `nltk`'s corpus. You can see the files names by running the code cell below.

In [2]:
gutenberg.fileids() # List of available documents in the Gutenberg corpus

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

To decrease the amount of text to be processed (your code would run for quite a long time if we used all of the above texts), we will just stick with the documents written by Jane Austen and William Shakespeare, each of which have 3 documents. The total of 6 texts are extracted for you in the following code cell and stored in a dictionary called `docs`.

`docs` is structured so that the file names are the keys and the values are the list of lowercased words associated with that file. The code prints the first 30 words from Shakespeare's *Hamlet* by getting the value associated with `"shakespeare-hamlet.txt"` and slicing the first 30 strings.

`sorted_doc_names` is also present in the next code cell. This is a list of the 6 document names (their file names) in sorted order. `sorted_doc_names` will be useful to reference later since **term-document matrices and TF-IDF matrices must have the documents (the columns) in a specific order**. A straightforward choice is sorted, alphabetical order.

In [3]:
### RUN THIS ###
docs = {}
for fileid in gutenberg.fileids():
  if fileid.startswith("shakespeare") or fileid.startswith("austen"): # Only get Shakespeare and Austen documents
    words = gutenberg.words(fileid)
    docs[fileid] = [word.lower() for word in words] # Associate the file name with the lowercased words from that file

sorted_doc_names = sorted(docs.keys()) # List of the document/file names in alphabetical order
print(sorted_doc_names)

hamlet_words = docs['shakespeare-hamlet.txt'] # Example: To access the words from Hamlet, use Hamlet file name
print(hamlet_words[:30]) # The associated value with that file name key is the list of words

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt']
['[', 'the', 'tragedie', 'of', 'hamlet', 'by', 'william', 'shakespeare', '1599', ']', 'actus', 'primus', '.', 'scoena', 'prima', '.', 'enter', 'barnardo', 'and', 'francisco', 'two', 'centinels', '.', 'barnardo', '.', 'who', "'", 's', 'there', '?']


# Question 1: Term-Document Matrix

Your first task is the construct a term-document matrix from our corpus, which is a subset of `nltk`'s selection of Gutenberg texts. Remember that a term-document matrix associates each row with a word/token and each column with a document. In this case, there will be 6 columns since we have 6 documents.

## Part (a)

The number of rows in the term-document matrix is determined by the size of the vocabulary. So before we can create the matrix, we have to compute the vocabulary.

Whereas a document may have a given word present multiple times, a vocabulary just lists each word once. What data structure would make the most sense to use to represent a vocabulary then? One solid option is a set. Remember that an empty set can be created using Python's built in `set()` function. Sets also have an `add(new_element)` function for adding new elements to the set.

Much like how `sorted_doc_names` will be useful for referencing which column corresponds to which document, we will also want to do the same for each row corresponding to the right word. The most straightforward ordering of strings is alphabetical order, plus we need to be able to index into the vocabulary. 

This leads to the following steps:

* **Extract the vocabulary of the corpus (all documents) as a set.** The point of using a set is to avoid duplicates.
* **Convert that set to a list and sort the list.** This variable will be useful later to know what word each row corresponds to.
* **Print the length of the list to see the size of the vocabulary.**

In [4]:
### YOUR CODE HERE ###
vocab_set = set()
for doc_name in sorted_doc_names:
    words = docs[doc_name]
    for word in words:
        vocab_set.add(word)

vocab_list = sorted(list(vocab_set))
print("Vocabulary size:", len(vocab_list))


Vocabulary size: 15683


## Part (b)

Now we can create the matrix. You could choose to use a list of lists here, but as you will see later, it is much more convenient to use `numpy`'s matrices and arrays. `numpy` is imported as `np`, as per convention.

`np.zeros(shape)` is a useful function for initializing arrays and matrices, namely with 0 for every value. To create a matrix, `shape` must be specified as a tuple with two values. For example, `np.zeros((2,3))` (note the parentheses) would create the matrix
```
array([[0., 0., 0.],
       [0., 0., 0.]])
```
The first value of the tuple for `shape` corresponds to the number of rows. The second value corresponds to the number of columns.

Of course, this matrix will not just contain zeros. The element at row `i`, column `j` of your term-document matrix should have the term frequency of the word corresponding to row `i` in the document corresponding to column `j`.

**Create the term-document matrix for our corpus of 6 documents. Print the matrix at the end to at least confirm it at least contains non-zero values.**

*This code cell will take the longest to run of everything in this notebook, maybe around 1-3 minutes depending on your connection. In fact, if you've ever used an application (e.g. an IDE like IntelliJ or Pycharm) that shows the word "Indexing..." when it's loading, it's because the application is doing something similar to this for your files.*

*Suggestion: To create the matrix, you will have to iterate over all the words and all the documents. It may be useful to have the outer loop correspond to the documents so you can print which document you're currently on and know how long it's taking to process.*

In [5]:
### YOUR CODE HERE ###
# Step 1: Initialize the numpy matrix
term_doc_matrix = np.zeros((len(vocab_list), len(sorted_doc_names)))

# Step 2: Iterate over each document
for j, doc_name in enumerate(sorted_doc_names):
    words = docs[doc_name]
    
    # Step 3: Iterate over each word in the document
    for word in words:
        # Step 4: Increment the corresponding element in the numpy matrix
        i = vocab_list.index(word)
        term_doc_matrix[i, j] += 1
    
    print(f"Processed document {j+1}/{len(sorted_doc_names)}")

# Step 5: Print the resulting matrix
print(term_doc_matrix)


Processed document 1/6
Processed document 2/6
Processed document 3/6
Processed document 4/6
Processed document 5/6
Processed document 6/6
[[549. 253. 289.  21.  17.   5.]
 [144.  61. 113.   0.   0.   0.]
 [ 16.   0.  13.   0.   0.   0.]
 ...
 [  0.   1.   1.   0.   0.   0.]
 [  1.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1.   0.]]


# Question 2: TF-IDF

Term-document matrices are not typically used because raw frequencies tend not to correspond to the actual amount of information for each word. A TF-IDF-weighted matrix, though, is often more accurate in this regard.

Remember that the TF-IDF value for any given term $t$ in document $d$ in our corpus of $N$ documents is a combination of term frequency and inverse document frequency:

$$
\text{tf}_{t,d} = \log(count(t,d)+1)
$$

$$
\text{idf}_{t} = \log(\frac{N}{df_t})
$$

$$
w_{t,d} = \text{tf}_{t,d} \times \text{idf}_{t}
$$

## Part (a)

Before getting into the code, here is a short-answer response question first. Notice that $\text{tf}_{t,d}$ uses add-one smoothing to prevent taking the log of 0, which is undefined. However, $\text{idf}_t$ does not add one (to the denominator nor the ratio) even though division by 0 is undefined.

**Why does the formula for $\textbf{idf}_t$ not require add-one smoothing? Answer in your own words in the following text cell.** *Hint: consider how the term-document matrix is constructed.*

YOUR ANSWER HERE

The formula for idft does not require add-one smoothing because the denominator of the idft formula, Ndft (i.e., the number of documents in the corpus containing the term t), can never be zero. This is because the term-document matrix is constructed in such a way that all words in the vocabulary are represented as rows, and each row has a non-zero count in at least one document (even if it's just the document in which the word appears). Therefore, every term in the vocabulary will have a non-zero Ndft value, making add-one smoothing unnecessary for the idft formula.

## Part (b)

**Create another matrix, based on your term-document matrix from the previous question to avoid re-reading all the documents, with TF-IDF weighted values instead of raw frequencies.** This one should not take long like the original matrix did since you're not processing based on the strings in each document.

*For the next question it is important that you have both the original term-document matrix and the TF-IDF matrix, so do not overwrite the matrix from above.*

In [7]:
### YOUR CODE HERE ###
import numpy as np

# load the term-document matrix from the previous question


# calculate IDF values
N = term_doc_matrix.shape[1]
idf = np.log(N / (1 + np.count_nonzero(term_doc_matrix, axis=1)))

# calculate TF-IDF values
tf_idf_matrix = np.zeros(term_doc_matrix.shape)
for i in range(term_doc_matrix.shape[0]):
    for j in range(term_doc_matrix.shape[1]):
        tf_idf_matrix[i][j] = term_doc_matrix[i][j] * idf[i]

# save the TF-IDF matrix for future use
np.save('tf_idf_matrix.npy', tf_idf_matrix)


# Question 3: Calculating Similarity

Here is where we put our matrices to use. The `numpy` matrices only contain the numerical values, not the associated words and document names, so to get a word or document vector we have an intermediate step. Parts (a) and (b) will address these by defining some useful functions.

## Part (a)

**Define a function called `word_vector` that takes in a word/token and a `numpy` matrix, and returns the row from the given matrix associated with the given word.** This will involve getting the word's index in the sorted list of vocabulary words, then indexing into the matrix. Indeed, you will be referencing a variable you created in the first question within this function.

**Print the word vector for `"the"` using the term-document matrix, then print the word vector for `"the"` using the TF-IDF matrix.** Although the results are both plausible word vectors for the same word, the values should be notably different in each and should confirm your intuition on what the results should be for a word like "the".

*Hint: Python lists have a built-in function called `index` that will give you the index of (the first instance of, when there are duplicates) the given element.*

In [13]:
### YOUR CODE HERE ###
def word_vector(word, matrix):
    index = vocab_list.index(word)
    return matrix[index]


## Part (b)

**Define a function called `doc_vector` that takes in a document/file name (as originally formatted with the `.txt` extension) and a `numpy` matrix, and returns the column from the given matrix associated with the given document.** This is where `numpy`'s arrays come in handy. Getting a column from a list of lists in basic Python is tricky. `numpy` makes it easy with the following syntax.

To get column `col` from a matrix `matrix`, you can use
```
matrix[:, col] # Outputs column col
```
This makes use of `numpy`'s slicing syntax, much like how you can slice a 1D list. The `:` means "give me every row" since the absence of a start and end row on either side of the `:` defaults to the start and end being 0 and `n` for `n` total rows.

**Print the document vector for `"shakespeare-hamlet.txt"` using the term-document matrix, then print the document vector for `"shakespeare-hamlet.txt"` using the TF-IDF matrix. Also print the length of either vector to confirm that it matches the size of the vocabulary computed previously.** 

In [10]:
### YOUR CODE HERE ###
def doc_vector(doc_name, matrix):
    index = sorted_doc_names.index(doc_name)
    return matrix[:, index]

# Using term-document matrix
hamlet_td = doc_vector('shakespeare-hamlet.txt', term_doc_matrix)
print("Term-document matrix vector for Hamlet:\n", hamlet_td)
print("Length of term-document matrix vector for Hamlet:", len(hamlet_td))

# Using TF-IDF matrix
hamlet_tfidf = doc_vector('shakespeare-hamlet.txt', tf_idf_matrix)
print("\nTF-IDF matrix vector for Hamlet:\n", hamlet_tfidf)
print("Length of TF-IDF matrix vector for Hamlet:", len(hamlet_tfidf))



Term-document matrix vector for Hamlet:
 [17.  0.  0. ...  0.  0.  1.]
Length of term-document matrix vector for Hamlet: 15683

TF-IDF matrix vector for Hamlet:
 [-2.62056156  0.          0.         ...  0.          0.
  1.09861229]
Length of TF-IDF matrix vector for Hamlet: 15683


## Part (c)

**Define a function called `sim` or `similarity` that takes in two vectors (1-dimensional `numpy` arrays) and returns the cosine similarity between the two vectors.** Remember that cosine similarity is defined as:

$$
\text{cos}(\theta) = \frac{\textbf{a}\cdot\textbf{b}}{|\textbf{a}||\textbf{b}|}
$$

$\textbf{a}\cdot\textbf{b}$ is the dot product between vectors $\textbf{a}$ and $\textbf{b}$. $|\textbf{a}|$ is the L2 norm of $\textbf{a}$ which can be easily calculated using `numpy.linalg`'s `norm` function.

**Get the word vectors for `"king"`, `"queen"`, and `"apple"` (or any other very different third word that appears in these documents) using your `word_vector` function based on the term-document matrix. Print the similarity between `"king"` and `"queen"`, then print the similarity between `"king"` and `"apple"` (or whichever obviously different word you choose) using your `similarity` function.** The results should be pretty intuitive given the word choice.

**Do the same thing using the TF-IDF vectors for each word to see how the results differ.** You should find that the TF-IDF-based similarities are more compressed than the term-document ones.

In [14]:
from numpy.linalg import norm

### YOUR CODE HERE ###
import numpy as np

def similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_product = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    return dot_product / norm_product

# Get word vectors for "king", "queen", and "apple"
king_vec = word_vector("king", term_doc_matrix)
queen_vec = word_vector("queen", term_doc_matrix)
apple_vec = word_vector("apple", term_doc_matrix)

# Compute similarities using the term-document matrix
print("Similarity between king and queen (term-document matrix):", similarity(king_vec, queen_vec))
print("Similarity between king and apple (term-document matrix):", similarity(king_vec, apple_vec))

# Get word vectors for "king", "queen", and "apple" using the TF-IDF matrix
king_vec_tfidf = word_vector("king", tf_idf_matrix)
queen_vec_tfidf = word_vector("queen", tf_idf_matrix)
apple_vec_tfidf = word_vector("apple", tf_idf_matrix)

# Compute similarities using the TF-IDF matrix
print("Similarity between king and queen (TF-IDF matrix):", similarity(king_vec_tfidf, queen_vec_tfidf))
print("Similarity between king and apple (TF-IDF matrix):", similarity(king_vec_tfidf, apple_vec_tfidf))


Similarity between king and queen (term-document matrix): 0.9319871710504082
Similarity between king and apple (term-document matrix): 0.011072051921490599
Similarity between king and queen (TF-IDF matrix): 0.9319871710504082
Similarity between king and apple (TF-IDF matrix): 0.011072051921490597


## Part (d)

This part will attempt to answer the question of *\"how similar are Jane Austen's works to William Shakespeare's works?\"* using our small corpus of 3 works for each author and the matrices you've constructed above.

To do so, you should get the document vectors for each of the documents and compute the **centroid vector** (or just "centroid") of the document vectors from Jane Austen and the centroid of the document vectors from Shakespeare. The centroid is the average of a set of vectors. For $k$ vectors:
$$
\text{centroid} = \frac{\textbf{v}_1+\textbf{v}_2+...+\textbf{v}_k}{k}
$$
where $+$ is element-wise addition between vectors and division by $k$ is element-wise division. Both operations are handled intuitively by `numpy` arrays.

**Compute the centroid for Jane Austen's works and the centroid for Shakespeare's works with the matrix of your choice. Compute and print the cosine similarity between the two centroids using your `similarity` function.**

*You do not have to do this for both types of vectors for this part (it can be a lot of redundant code). If you do print both, you should find the results are very different.*

In [30]:
def search_docs(keywords, source, matrix):
    """
    Searches for documents containing all the given keywords using the given matrix.
    
    Parameters:
        keywords (tuple): A tuple of strings representing the keywords to search for.
        source (list): A list of strings representing the file names to search through.
        matrix (numpy.ndarray): A numpy array representing the term-document or TF-IDF matrix.
    
    Returns:
        A tuple of numpy arrays representing the matches for each keyword.
    """
    # Initialize empty arrays for each keyword
    matches = tuple(np.array([]) for _ in keywords)
    
    # Loop through each file in the source list
    for file in source:
        # Get the document vector for the file
        doc_vec = doc_vector(file, matrix)
        
        # Check if all keywords are present in the document
        if all(keyword in vocab_list for keyword in keywords):
            # Loop through each keyword and add the matching entries to the corresponding array
            for i, keyword in enumerate(keywords):
                keyword_index = vocab_list.index(keyword)
                keyword_matches = doc_vec[keyword_index]
                matches[i] = np.append(matches[i], keyword_matches)
    
    return matches



import numpy as np

# Get the document vectors for Shakespeare and Jane Austen works
shakespeare_docs = [doc for doc in docs if 'shakespeare' in doc.lower()]
ja_docs = [doc for doc in docs if 'austen' in doc.lower()]

shakespeare_term_doc_vectors = [doc_vector(doc, term_doc_matrix) for doc in shakespeare_docs]
ja_term_doc_vectors = [doc_vector(doc, term_doc_matrix) for doc in ja_docs]

shakespeare_tf_idf_vectors = [doc_vector(doc, tf_idf_matrix) for doc in shakespeare_docs]
ja_tf_idf_vectors = [doc_vector(doc, tf_idf_matrix) for doc in ja_docs]


# Compute the centroid vector for Shakespeare works and Jane Austen works
shakespeare_term_doc_centroid = np.mean(shakespeare_term_doc_vectors, axis=0)
ja_term_doc_centroid = np.mean(ja_term_doc_vectors, axis=0)

shakespeare_tf_idf_centroid = np.mean(shakespeare_tf_idf_vectors, axis=0)
ja_tf_idf_centroid = np.mean(ja_tf_idf_vectors, axis=0)

# Compute the cosine similarity between the centroid vectors using the term-document vectors
similarity_term_doc = similarity(shakespeare_term_doc_centroid, ja_term_doc_centroid)

# Compute the cosine similarity between the centroid vectors using the TF-IDF vectors
similarity_tf_idf = similarity(shakespeare_tf_idf_centroid, ja_tf_idf_centroid)

# Print the cosine similarities
print(f"Cosine similarity using term-document vectors: {similarity_term_doc}")
print(f"Cosine similarity using TF-IDF vectors: {similarity_tf_idf}")


Cosine similarity using term-document vectors: 0.8885801784399422
Cosine similarity using TF-IDF vectors: 0.7587857384811342
